Meeting notes:

##### Make diag.bib update code (notebook)
- Compare titles of bib and semantic scholar
- Match
- If match:
    - Add semantic scholar ID to diag bib
    - Update gscites field from bib entry
- If no match:
    - Check best matches, if DOI and Arxiv --> chose DOI 
    - Define right fields (look at code James) + citations + semantic scolar ID 
    - Add 'automatically added' field so we can trace back automation errors more easilty
    - Add to bib   
    
##### Make sript that checks ss ids and adds new ones to diag bib
    
    
##### Make script that can be used by DIAG members
- Add doi/arxiv
- if DOI check if Arxiv exists, if so --> remove Arxiv
- Sort right fields from DOI --> add fields
- Search for semantic scholar ID
- Print fields so user can check it
- Add to bib
- Manual commit push

##### Make script that checks all semantic scholar IDs and updates gscites
- github action overnight?

##### Note: 
- requests limit, 100 requests per 5 mins for ss 
    - 100 papers at once = 1 request
- google doc: 
    - https://docs.google.com/document/d/1uFyh5Z-wieksq_4Bf8Br7wHpN-buHECunS0VurYnlvg/edit


# Libraries

In [1]:
import os
import numpy as np
import pandas as pd

import requests
# from bibreader import parse_bibtex_file, get_bib_blocks
from bib_handling_code.processbib import read_bibfile
from bib_handling_code.processbib import save_to_file
from difflib import SequenceMatcher

# Example from url giving 100 papers from one of Bram's semantic scolar IDs

In [2]:
staff_id = 123637526 #8038506
staff_ids = [123637526]

In [3]:
url=f'https://api.semanticscholar.org/graph/v1/author/{staff_id}/papers?fields=year,title,authors,externalIds,citationCount&limit=100'

r = requests.get(url)
ss_staff_data = r.json()['data']

In [4]:
for i in ss_staff_data:
    if i['paperId'] == 'c70d5169b5c69171cc2ed3ec40eebda0630988d9':
        display(i)

{'paperId': 'c70d5169b5c69171cc2ed3ec40eebda0630988d9',
 'externalIds': {'DOI': '10.1038/s41598-020-62148-y', 'CorpusId': 257025991},
 'title': 'Computer aided detection of tuberculosis on chest radiographs: An evaluation of the CAD4TB v6 system',
 'year': 2020,
 'citationCount': 0,
 'authors': [{'authorId': '35730362', 'name': 'K. Murphy'},
  {'authorId': '6214077', 'name': 'S. Habib'},
  {'authorId': '40347668', 'name': 'S. M. Zaidi'},
  {'authorId': '39726137', 'name': 'S. Khowaja'},
  {'authorId': '2208892980', 'name': 'Aamir Khan'},
  {'authorId': '144861586', 'name': 'J. Melendez'},
  {'authorId': '152867460', 'name': 'E. Scholten'},
  {'authorId': '81054906', 'name': 'Farhan Amad'},
  {'authorId': '3504603', 'name': 'S. Schalekamp'},
  {'authorId': '40442474', 'name': 'Maurits Verhagen'},
  {'authorId': '38557856', 'name': 'R. Philipsen'},
  {'authorId': '2074941083', 'name': 'Annet Meijers'},
  {'authorId': '123637526', 'name': 'B. van Ginneken'}]}

# Function to get data from DOI/ArXiv

In [5]:
def get_doi_csl(doi):
    response = requests.get(
        f"https://doi.org/{doi}",
        headers={"Accept": "application/vnd.citationstyles.csl+json"},
        timeout=5,
    )
    response.raise_for_status()
    return response.json()

# ####################
# Example
# ####################

In [6]:
get_doi_csl('10.48550/arXiv.2302.01738').keys()

dict_keys(['type', 'id', 'categories', 'author', 'issued', 'abstract', 'DOI', 'publisher', 'title', 'URL', 'copyright', 'version'])

In [7]:
ss_staff_data[2]['externalIds']

{'PubMedCentral': '9142841',
 'DOI': '10.1117/1.JMI.9.5.052407',
 'CorpusId': 249160972,
 'PubMed': '35692896'}

In [8]:
staff_id = 8038506
url=f'https://api.semanticscholar.org/graph/v1/author/{staff_id}/papers?fields=year,title,authors,externalIds,citationCount&limit=100'

r = requests.get(url)
ss_staff_data = r.json()['data']

In [9]:
found_dois = []

In [10]:
found_arxiv_doi = False
found_regular_doi = False

In [11]:
#ArXiv
for ss_staff_entry in ss_staff_data:
    if 'DOI' in ss_staff_entry['externalIds']:
        if 'arXiv' in ss_staff_entry['externalIds']['DOI']:
            arxiv_doi = ss_staff_entry['externalIds']['DOI']
            found_arxiv_doi = True
    if found_arxiv_doi:
        break
#DOI
for ss_staff_entry in ss_staff_data:
    if 'DOI' in ss_staff_entry['externalIds']:
        if 'arXiv' not in ss_staff_entry['externalIds']['DOI']:
            regular_doi = ss_staff_entry['externalIds']['DOI']
            
            found_regular_doi = True
    if found_regular_doi:
        break

print(arxiv_doi)
print(regular_doi)

10.48550/arXiv.2302.01790
10.1016/j.media.2023.102755


In [12]:
arxiv_doi_data = get_doi_csl(arxiv_doi)
regular_doi_data = get_doi_csl(regular_doi)

In [13]:
for ss in ss_staff_data:
    if 'type' in ss:
        print(ss['type'])

In [14]:
[print(key) for key in arxiv_doi_data.keys()];

type
id
categories
author
issued
abstract
DOI
publisher
title
URL
copyright
version


In [15]:
arxiv_doi_data

{'type': 'article',
 'id': 'https://doi.org/10.48550/arxiv.2302.01790',
 'categories': ['Computer Vision and Pattern Recognition (cs.CV)',
  'FOS: Computer and information sciences',
  'FOS: Computer and information sciences'],
 'author': [{'family': 'Reinke', 'given': 'Annika'},
  {'family': 'Tizabi', 'given': 'Minu D.'},
  {'family': 'Baumgartner', 'given': 'Michael'},
  {'family': 'Eisenmann', 'given': 'Matthias'},
  {'family': 'Heckmann-Nötzel', 'given': 'Doreen'},
  {'family': 'Kavur', 'given': 'A. Emre'},
  {'family': 'Rädsch', 'given': 'Tim'},
  {'family': 'Sudre', 'given': 'Carole H.'},
  {'family': 'Acion', 'given': 'Laura'},
  {'family': 'Antonelli', 'given': 'Michela'},
  {'family': 'Arbel', 'given': 'Tal'},
  {'family': 'Bakas', 'given': 'Spyridon'},
  {'family': 'Benis', 'given': 'Arriel'},
  {'family': 'Blaschko', 'given': 'Matthew'},
  {'family': 'Büttner', 'given': 'Florian'},
  {'family': 'Cardoso', 'given': 'M. Jorge'},
  {'family': 'Cheplygina', 'given': 'Veronika'},

In [16]:
[print(key) for key in regular_doi_data.keys()];

indexed
reference-count
publisher
license
content-domain
published-print
DOI
type
created
page
update-policy
source
is-referenced-by-count
title
prefix
volume
author
member
reference
container-title
original-title
language
link
deposited
score
resource
subtitle
short-title
issued
references-count
alternative-id
URL
relation
ISSN
subject
container-title-short
published
assertion
article-number


In [17]:
regular_doi_data

{'indexed': {'date-parts': [[2023, 2, 25]],
  'date-time': '2023-02-25T05:29:46Z',
  'timestamp': 1677302986187},
 'reference-count': 38,
 'publisher': 'Elsevier BV',
 'license': [{'start': {'date-parts': [[2023, 4, 1]],
    'date-time': '2023-04-01T00:00:00Z',
    'timestamp': 1680307200000},
   'content-version': 'tdm',
   'delay-in-days': 0,
   'URL': 'https://www.elsevier.com/tdm/userlicense/1.0/'},
  {'start': {'date-parts': [[2023, 2, 6]],
    'date-time': '2023-02-06T00:00:00Z',
    'timestamp': 1675641600000},
   'content-version': 'vor',
   'delay-in-days': 0,
   'URL': 'http://creativecommons.org/licenses/by/4.0/'}],
 'content-domain': {'domain': ['elsevier.com', 'sciencedirect.com'],
  'crossmark-restriction': True},
 'published-print': {'date-parts': [[2023, 4]]},
 'DOI': '10.1016/j.media.2023.102755',
 'type': 'journal-article',
 'created': {'date-parts': [[2023, 1, 25]],
  'date-time': '2023-01-25T12:21:08Z',
  'timestamp': 1674649268000},
 'page': '102755',
 'update-poli

# ####################################
#  FINAL
# ####################################

In [18]:
# df_bib_data = pd.read_csv('script_data/temp_diag_bib_summary.csv')
# df_all_staff_id_ss_data = pd.read_csv('script_data/temp_semantic_scholar_all_staff_id_last_100_summary.csv')
# df_doi_match_data = pd.read_csv('script_data/temp_doi_match_data.csv')
# df_title_match_data = pd.read_csv('script_data/temp_title_match_data.csv')

In [19]:
# df_bib_data

In [20]:
# df_all_staff_id_ss_data

In [21]:
# df_doi_match_data

In [22]:
# df_title_match_data

# Import diag.bib

In [23]:
path_diag_bib = os.path.join('..', 'diag.bib')
diag_bib_raw = read_bibfile(None, path_diag_bib) # I changed the code in such a way that IF I give a second argument, it uses the second argument as a full path

In [24]:
len(diag_bib_raw)

2810

In [25]:
diag_bib_raw[-1].fields

{'author': '{Vendittelli, Pierpaolo and Bokhorst, John-Melle and Smeets, Esther Markus and Kryklyva, Valentyna and Brosens, Lodewijk and Verbeke, Caroline and Litjens, Geert}',
 'booktitle': 'MIDL',
 'title': '{Automatic quantification of {TSR} as a prognostic marker for pancreatic cancer.}',
 'url': '{https://openreview.net/forum?id=Dtz_iaUpGc}',
 'abstract': "{The current diagnostic and outcome prediction methods for pancreatic cancer lack prognostic power. As such, identifying novel biomarkers using machine learning has become of increasing interest. In this study, we introduce a novel method for estimating the tumor-stroma ratio (TSR) in whole slide images (WSIs) of pancreatic tissue and assess its potential as a prognostic biomarker. A multi-step strategy for estimating TSR is proposed, including epithelium segmentation based on an immunohistochemical reference standard, a coarse pancreatic cancer segmentation, and a post-processing pipeline for TSR quantification. The resultant s

In [26]:
bib_data = []
for bib_entry in diag_bib_raw:
    bibkey = bib_entry.key
    bib_type = bib_entry.type
    
    bib_authors = None
    bib_title = None
    bib_doi = None
    bib_gscites = None
    bib_journal = None
    
    if 'author' in bib_entry.fields:
        bib_authors = bib_entry.fields['author'].strip('{ }')
    if 'title' in bib_entry.fields:
        bib_title = bib_entry.fields['title'].strip('{ }')
    if 'doi' in bib_entry.fields:
        bib_doi = bib_entry.fields['doi'].strip('{ }')
    if 'gscites' in bib_entry.fields:
        bib_gscites = bib_entry.fields['gscites'].strip('{ }')
    if 'journal' in bib_entry.fields:
        bib_journal = bib_entry.fields['journal'].strip('{ }')
    
    bib_data.append([bibkey, bib_type, bib_title, bib_authors, bib_doi, bib_gscites, bib_journal])
bib_columns = ['bibkey', 'type', 'title', 'authors', 'doi', 'gs_citations', 'journal']
df_bib_data = pd.DataFrame(bib_data, columns=bib_columns)

In [27]:
df_bib_data.to_csv('script_data/temp_diag_bib_summary.csv', index=False)

In [28]:
df_bib_data.tail()

,bibkey,type,title,authors,doi,gs_citations,journal
2805,Eeke23,article,Artificial intelligence in bone marrow histolo...,"van Eekelen, Leander and Litjens, Geert and He...",10.1159/000529701,None,PATHOB
2806,Linm23,article,Predictive uncertainty estimation for out-of-d...,"Linmans, Jasper and Elfwing, Stefan and van de...",10.1016/j.media.2022.102655,None,MIA
2807,Rein23,article,Understanding metric-related pitfalls in image...,"Reinke, Annika and Tizabi, Minu D. and Baumgar...",None,None,arXiv:2302.01790
2808,Jark22,article,Generalization of Deep Learning in Digital Pat...,"Jarkman, Sofia and Karlberg, Micael and Pocevi...",10.3390/cancers14215424,None,CANCERS
2809,Vend23,inproceedings,Automatic quantification of {TSR} as a prognos...,"Vendittelli, Pierpaolo and Bokhorst, John-Mell...",None,None,None


Used for title matching

In [29]:
df_bib_data_with_titles = df_bib_data[df_bib_data['title'].notna()]
df_bib_data_with_titles.head()

,bibkey,type,title,authors,doi,gs_citations,journal
1069,Abas05,article,Analysis of regularity in the {EEG} background...,D. Ab\'{a}solo and R. Hornero and P. Espino an...,10.1016/j.clinph.2005.04.001,233,CLINNP
1070,Genu22,conference,Imaging tumor-infiltrating CD8 (+) T-cells in ...,E. A. J. van Genugten and B. Piet and G. Schre...,None,None,None
1071,Abas05a,inproceedings,EEG} background activity analysis in {A}lzheim...,D. Ab\'{a}solo and C. G\'{o}mez and J. Poza an...,None,3,None
1072,Abel19,article,"Computational pathology definitions, best prac...","Abels, Esther and Pantanowitz, Liron and Aeffn...",10.1002/path.5331,24,JPAT
1073,Abra08a,article,Evaluation of a system for automatic detection...,M. D. Abr\`amoff and M. Niemeijer and M. S. A....,10.2337/dc07-1312,287,DC


# Gather ss and bib data for all staff member IDs

In [32]:
staff_dict = {'Bram van Ginneken': [8038506, 123637526],
'Francesco Ciompi': [143613202],
'Alessa Hering': [153744566],
'Henjan Huisman': [34754023],
'Colin Jacobs': [2895994],
'Peter Koopmans': [34726383],
'Jeroen van der Laak': [145441238, 145388932],
'Geert Litjens': [145959882],
'James Meakin': [4960344],
'Keelin Murphy': [35730362],
'Ajay Patel': [2109170880, 2116215861],
'Cornelia Schaefer-Prokop': [1419819133, 1445069528],
'Matthieu Rutten': [2074975080, 2156546],
'Jos Thannhauser': [5752941]}

In [33]:
staff_ids = []
[staff_ids.extend(values) for values in staff_dict.values()];
staff_ids

[8038506,
 123637526,
 143613202,
 153744566,
 34754023,
 2895994,
 34726383,
 145441238,
 145388932,
 145959882,
 4960344,
 35730362,
 2109170880,
 2116215861,
 1419819133,
 1445069528,
 2074975080,
 2156546,
 5752941]

## Semantic scolar

In [35]:
all_staff_id_ss_data = []

# get data for staff memb|er
for idx_staff_id, staff_id in enumerate(staff_ids):
    print(f'[{idx_staff_id+1}/{len(staff_ids)}]: {staff_id}')
    staff_id_ss_data = []
    # get semantic scolar data
    url=f'https://api.semanticscholar.org/graph/v1/author/{staff_id}/papers?fields=year,title,authors,externalIds,citationCount&limit=500'
    r = requests.get(url)
    ss_staff_data = r.json()['data']
    
    # loop over semantic scolar data entries
    for ss_staff_entry in ss_staff_data:
        ss_id = None
        ss_title = None
        ss_doi = None
        ss_citations = None

        if 'paperId' in ss_staff_entry:
            ss_id = ss_staff_entry['paperId']
        if 'title' in ss_staff_entry:
            ss_title = ss_staff_entry['title']
        if 'DOI' in ss_staff_entry['externalIds']:
            ss_doi = ss_staff_entry['externalIds']['DOI']
        if 'citationCount' in ss_staff_entry:
            ss_citations = ss_staff_entry['citationCount']

        staff_id_ss_data.append([staff_id, ss_id, ss_title, ss_doi, ss_citations])
    all_staff_id_ss_data.extend(staff_id_ss_data)    
ss_columns = ['staff_id', 'ss_id', 'title', 'doi', 'ss_citations']
df_all_staff_id_ss_data = pd.DataFrame(all_staff_id_ss_data, columns=ss_columns)
print('DONE')

[1/19]: 8038506
[2/19]: 123637526
[3/19]: 143613202
[4/19]: 153744566
[5/19]: 34754023
[6/19]: 2895994
[7/19]: 34726383
[8/19]: 145441238
[9/19]: 145388932
[10/19]: 145959882
[11/19]: 4960344
[12/19]: 35730362
[13/19]: 2109170880
[14/19]: 2116215861
[15/19]: 1419819133
[16/19]: 1445069528
[17/19]: 2074975080
[18/19]: 2156546
[19/19]: 5752941
DONE


In [36]:
df_all_staff_id_ss_data

,staff_id,ss_id,title,doi,ss_citations
0,8038506,1898998f27ce0750a42f8f3ca2ed7e292f05aee2,Understanding metric-related pitfalls in image...,10.48550/arXiv.2302.01790,5
1,8038506,362c510dec0d566d22d5be3af0519fc7eec8bb86,Uncertainty-Aware Multiple-Instance Learning f...,None,0
2,8038506,4bcd672218ecec70473c84f6f1cc52c64031f3e5,Continual learning strategies for cancer-indep...,10.1016/j.media.2023.102755,0
3,8038506,7981606bf8110ec6cc64baa22d694096f7862939,AIROGS: Artificial Intelligence for RObust Gla...,10.48550/arXiv.2302.01738,2
4,8038506,e738d94746b0f33b1c48737022ab186618d4d4cd,SimpleRad: Patient-Friendly Dutch Radiology Re...,10.1007/978-3-031-28241-6_18,0
...,...,...,...,...,...
1818,5752941,7971250a0b35dd47790d09e0c1d1fdcd16c5e802,Future directions in resuscitation care: Towar...,10.1016/J.RESUSCITATION.2018.07.350,0
1819,5752941,3cfe71ef252b69aeaa7d5e70b84afb518e253e82,Ventricular fibrillation waveform characterist...,10.1016/j.resuscitation.2017.03.029,8
1820,5752941,183eca01f54b6aca310e7d68a48a1bd6724745dd,Abstract 19221: Inter Shock Changes of the Ven...,None,0
1821,5752941,1b1b6d1c122164f90cecfa7150becf536a99b92d,Abstract 19090: Ventricular Fibrillation Wavef...,None,0


In [37]:
df_all_staff_id_ss_data[df_all_staff_id_ss_data['ss_id']=='90a870cbb9897124193ba18c5358fe45b6260621']

,staff_id,ss_id,title,doi,ss_citations
375,123637526,90a870cbb9897124193ba18c5358fe45b6260621,Feasibility of end-to-end trainable two-stage ...,10.1117/12.2551331,3
920,2895994,90a870cbb9897124193ba18c5358fe45b6260621,Feasibility of end-to-end trainable two-stage ...,10.1117/12.2551331,3


In [38]:
df_all_staff_id_ss_data.to_csv('script_data/temp_semantic_scholar_all_staff_id_last_100_summary.csv', index=False)

# Loop for matching DOIs and titles

In [42]:
temp_dois = df_all_staff_id_ss_data['doi'].dropna()

In [44]:
temp_dois[temp_dois.str.startswith('10')]

Series([], Name: doi, dtype: object)

In [49]:
incorrect_dois = []

In [47]:
from collections import Counter

In [71]:
doi_match_data = []
match_keys_columns = ['bibkey', 'ss_id', 'comment']
print('[MATCHING ON DOIS]')
# LOOP OVER BIB
for idx_bib, row_bib in df_bib_data.iterrows():
    bibkey = row_bib['bibkey']
    comment = None
    
    bib_doi = row_bib['doi']
    if bib_doi == None:
        continue
    index = bib_doi.find('10.')
    bib_doi = bib_doi[index:]
    doi_matching = df_all_staff_id_ss_data['doi'].isin([bib_doi])
    
    if doi_matching.any():
        matches = df_all_staff_id_ss_data[doi_matching] # could be multiple in SS, search for "Epithelium segmentation using deep learning" paper from Wouter Bulten... 
        if len(Counter(matches['ss_id'])) == 1:
            comment = 'single doi match'
            match = matches.iloc[0]
        if len(Counter(matches['ss_id'])) > 1:
            comment = f'multiple doi matches {list(Counter(matches["ss_id"]).keys())}, took the first one with most citations'
            match = matches[matches['ss_citations'] == matches['ss_citations'].max()].iloc[0]
        ss_id = match['ss_id']
        doi_match_data.append([bibkey, ss_id, comment])

df_doi_match_data = pd.DataFrame(doi_match_data, columns = match_keys_columns)
df_doi_match_data.to_csv('script_data/temp_doi_match_data.csv', index=False)
print('DONE')
print(len(df_doi_match_data['comment'].notna()))
print(len(df_doi_match_data))

[MATCHING ON DOIS]
DONE
621
621


In [72]:
np.unique(df_doi_match_data['comment'], return_counts=True)[1]

array([  1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1, 599], dtype=int64)

In [73]:
len(incorrect_dois)

23

In [37]:
import time

In [38]:
start = time.time()
print('[MATCHING ON TITLES]')
title_match_data = []
for idx_ss, row_ss in df_all_staff_id_ss_data.iterrows():
    if idx_ss % 50 == 0:
        print(f'[{idx_ss}/{len(df_all_staff_id_ss_data)}]')
    
    ss_id = row_ss['ss_id']
    # if ss_id in df_doi_match_data['ss_id']:
    #     continue
 
    ss_title = row_ss['title']
    if pd.isna(ss_title):
        continue
    
    title_match_ratios = df_bib_data_with_titles['title'].apply(lambda x: SequenceMatcher(
        a=ss_title.lower(), 
        b=x.lower().replace('{', '').replace('}', '')).ratio())

    max_ratio = title_match_ratios.max()
    max_bibkey = df_bib_data_with_titles[title_match_ratios==max_ratio]['bibkey'].iloc[0]
    max_bib_title = df_bib_data_with_titles[title_match_ratios==max_ratio]['title'].iloc[0]
    max_bib_title = max_bib_title.replace('{', '').replace('}', '')
    
    up80_bibkeys = None
    up80_bib_titles = None
    up80_ratios = None
    if sum(title_match_ratios>0.8) > 1:
        
        up80_bibkeys = list(df_bib_data_with_titles[title_match_ratios>0.8]['bibkey'])
        up80_bib_titles = list(df_bib_data_with_titles[title_match_ratios>0.8]['title'])
        up80_bib_titles = [up80_bib_title.replace('{', '').replace('}', '') for up80_bib_title in up80_bib_titles]
        up80_ratios = list(title_match_ratios[title_match_ratios>0.8])
    
    title_match_data.append([ss_id, ss_title, 
                             max_bibkey, max_bib_title, max_ratio, 
                             up80_bibkeys, up80_bib_titles, up80_ratios])

title_matches_columns = ['ss_id', 'ss_title', 
                         'max_bibkey' , 'max_bib_title', 'max_ratio',
                         'up80_bibkeys' , 'up80_bib_titles', 'up80_ratios',
                        ]
df_title_match_data = pd.DataFrame(title_match_data, columns=title_matches_columns)
df_title_match_data.to_csv('script_data/temp_title_match_data.csv', index=False)
print('DONE')
end = time.time()
sec = end-start
print(sec, 'sec')
print(sec/60, 'min')

[MATCHING ON TITLES]
[0/1112]
[50/1112]
[100/1112]
[150/1112]
[200/1112]
[250/1112]
[300/1112]
[350/1112]
[400/1112]
[450/1112]
[500/1112]
[550/1112]
[600/1112]
[650/1112]
[700/1112]
[750/1112]
[800/1112]
[850/1112]
[900/1112]
[950/1112]
[1000/1112]
[1050/1112]
[1100/1112]
DONE
750.2382960319519 sec
12.503971600532532 min


In [39]:
[ss_id, ss_title, max_bibkey, max_bib_title, max_ratio, up80_bibkeys, up80_bib_titles, up80_ratios]

['4cc849616719e9989d22a8fb586231762579f08c',
 'Ventricular fibrillation waveform characteristics differ according to the presence of a previous myocardial infarction: A surface ECG study in ICD-patients.',
 'Gall13',
 'Normalization of CT scans reconstructed with different kernels to reduce variability in emphysema measurements',
 0.42857142857142855,
 None,
 None,
 None]

In [40]:
df_bib_data.iloc[1086]

bibkey                                      Anto21
type                                       article
title           The Medical Segmentation Decathlon
doi                                           None
gs_citations                                  None
journal            arXiv preprint arXiv:2106.05735
Name: 1086, dtype: object

In [41]:
df_bib_data.iloc[1087]

bibkey                                           Anto22
type                                            article
title           The {Medical} {Segmentation} {Decathlon
doi                          10.1038/s41467-022-30695-9
gs_citations                                       None
journal                                          NATCOM
Name: 1087, dtype: object

In [42]:
max_ratio = title_match_ratios.max()
max_bibkey = df_bib_data_with_titles[title_match_ratios==max_ratio]['bibkey'].iloc[0]
max_bib_title = df_bib_data_with_titles[title_match_ratios==max_ratio]['title'].iloc[0]
max_bib_title = max_bib_title.replace('{', '').replace('}', '')


up80_bibkeys = None
un80_bib_titles = None


In [43]:
[ss_id, bibkey, max_ratio, bib_title, ss_title]

['4cc849616719e9989d22a8fb586231762579f08c',
 'Wild23b',
 0.42857142857142855,
 'Medical diffusion on a budget: textual inversion for medical image generation',
 'Ventricular fibrillation waveform characteristics differ according to the presence of a previous myocardial infarction: A surface ECG study in ICD-patients.']

In [44]:
df_bib_data_with_titles[title_match_ratios==max_ratio]['bibkey']

1403    Gall13
Name: bibkey, dtype: object

In [45]:
bib_title

'Medical diffusion on a budget: textual inversion for medical image generation'

In [46]:
SequenceMatcher(a='AAaa', b="aaaa").ratio()

0.5

In [47]:
df_title_matches[df_title_matches['max_ratio']<=0.8].sort_values('max_ratio')

NameError: name 'df_title_matches' is not defined

In [ ]:
df_title_matches[df_title_matches['max_ratio']>0.8]

In [ ]:
df_bib_data[title_match_ratios==max_ratio]

In [ ]:
row_ss

In [ ]:
df_bib_data[df_bib_data['title'].notna()]

In [ ]:
len(df_match_keys)

In [ ]:
df_title_matches

In [ ]:
# df_match_keys

In [ ]:
'Anto22' in df_match_keys['bibkey']

In [ ]:
df_staff_id_ss_data[doi_matching]['ss_citations'].max()

In [ ]:
df_staff_id_ss_data[doi_matching][df_staff_id_ss_data[doi_matching]['ss_citations'] == df_staff_id_ss_data[doi_matching]['ss_citations'].max()].iloc[0]

In [ ]:
df_staff_id_ss_data['doi']

In [ ]:
df_staff_id_ss_data[doi_matching]['ss_id'].item()

In [ ]:
row_bib

In [ ]:
row_ss['title']

In [ ]:
df_staff_id_ss_data['doi'].isin([row_bib['doi']]).any()

In [ ]:
pd.isin(row_bib['doi'], df_staff_id_ss_data['doi'])

In [ ]:
bib_title = bib_entry.fields['title'].strip('{ }')
ss_title = ss_entry

In [ ]:
all_types = []
for entry in diag_bib_raw:
    all_types.append(entry.type)
np.unique(all_types, return_counts=True)

In [ ]:
selected_entries = []
for entry in diag_bib_raw:
    if entry.type in ['article']: #['article', 'book', 'conference', 'inbook', 'inproceedings']
        selected_entries.append(entry)
        print(entry.fields)

In [ ]:
[print(key) for key in selected_entries[0].fields.keys()];